In [0]:
import torchvision
from torchvision import transforms, datasets, models
import torch
from torch import optim, cuda
from torch.utils.data import DataLoader, sampler
import torch.nn as nn
from PIL import Image
import os
import matplotlib.pyplot as plt
import numpy as np

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


/*IMAGE PROCESSING */

In [0]:
labels = np.load('/content/drive/Shared drives/Data/Reprocessed data /augmentedLabels.npy',allow_pickle=True)
training = np.load('/content/drive/Shared drives/Data/Reprocessed data /augmentedTrainingData.npy',allow_pickle=True)

In [0]:
#flips the numpy because tensorflow has channels at the end and pytorch has channels at the beginning
training = np.swapaxes(training,1,3)
training = np.swapaxes(training,2,3)

// Turns numpy arrays in to tensors

In [0]:
training = torch.from_numpy(training)

In [0]:
# just dummy data
training = torch.randn((100, 3, 224, 224))
labels = torch.zeros((100))
labels[50:] = 1

//Turns labels form one HOT encoding to binary

In [0]:
# I don't believe that pytorch supports one hot encoding for the y_training data so changing it (I could be wrong)
labels_temp = np.zeros(labels.shape[0])

for i in range(0,labels_temp.shape[0]): 
  if labels[i][0] == 1:
    labels_temp[i] = 1
  else:
    labels_temp[i] = 0

labels = torch.from_numpy(labels_temp)

labels

In [0]:
labels = labels.type(torch.LongTensor)
labels

In [0]:
#Check to see if we have GPU availables
print(cuda.is_available())
print(cuda.device_count())

In [0]:
device = torch.device('cuda:0')

/* END OF PROCESSING IMAGES, TIME FOR TRAINING */

## Self Attention Module

In [0]:
# https://github.com/heykeetae/Self-Attention-GAN/blob/master/sagan_models.py
class Self_Attn(nn.Module):
    """ Self attention Layer"""
    def __init__(self,in_dim):
        super(Self_Attn,self).__init__()
        self.chanel_in = in_dim
        
        self.query_conv = nn.Conv2d(in_channels = in_dim , out_channels = in_dim//8 , kernel_size= 1)
        self.key_conv = nn.Conv2d(in_channels = in_dim , out_channels = in_dim//8 , kernel_size= 1)
        self.value_conv = nn.Conv2d(in_channels = in_dim , out_channels = in_dim , kernel_size= 1)
        self.gamma = nn.Parameter(torch.zeros(1))

        self.softmax  = nn.Softmax(dim=-1) #
    def forward(self,x):
        """
            inputs :
                x : input feature maps( B X C X W X H)
            returns :
                out : self attention value + input feature 
                attention: B X N X N (N is Width*Height)
        """
        m_batchsize,C,width ,height = x.size()
        proj_query  = self.query_conv(x).view(m_batchsize,-1,width*height).permute(0,2,1) # B X CX(N)
        proj_key =  self.key_conv(x).view(m_batchsize,-1,width*height) # B X C x (*W*H)
        energy =  torch.bmm(proj_query,proj_key) # transpose check
        attention = self.softmax(energy) # BX (N) X (N) 
        proj_value = self.value_conv(x).view(m_batchsize,-1,width*height) # B X C X N

        out = torch.bmm(proj_value,attention.permute(0,2,1) )
        out = out.view(m_batchsize,C,width,height)
        
        out = self.gamma*out + x
        return out,attention

In [0]:
class SAResNet(nn.Module):

  def __init__(self, 
               model, # original ResNet, 
               module,
               device): # which layer to insert self attention
    super(SAResNet,self).__init__()
    self.model = model
    self.module = module
    self.device = device
    self.layers = [
        "conv1",
        "bn1",
        "relu",
        "maxpool",
        "layer1",
        "layer2",
        "layer3",
        "layer4",
        "avgpool"
    ]
    assert module in self.layers
    self.attention = None
    
  def forward(self, x):
    for layer in self.layers:
      x = self.model._modules[layer](x) # equivalent to self.model.layer(x)
      # We know to apply self attention here
      if layer == self.module:
        if self.attention is None:
          self.attention = Self_Attn(x.shape[1])
          self.attention.to(device)
        x, attention_map = self.attention(x)
    x = torch.flatten(x, 1)
    x = self.model.fc(x)
    return x, attention_map


In [0]:
resnet18 = models.resnet18(pretrained=True)

In [0]:
n_classes = 2
resnet18.fc = torch.nn.Linear(in_features = resnet18.fc.in_features, out_features=n_classes)
resnet18.to(device)

### Sanity Check Resnet Reimplementation

In [0]:
resnet18(training[:2].to(device))

tensor([[-0.0420,  0.1581],
        [ 0.4885,  0.3557]], device='cuda:0', grad_fn=<AddmmBackward>)

In [0]:
sa_model = SAResNet(resnet18, "layer1", device)

In [0]:
x, attn_map = sa_model(training[:2].to(device))

In [0]:
attn_map.shape

torch.Size([2, 3136, 3136])

In [0]:
resnet18._modules['conv1']

Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

In [0]:
## Data Augmentations

In [0]:
class CustomTensorDataset(torch.utils.data.Dataset):
    """TensorDataset with support of transforms.
    """
    def __init__(self, tensors, labels, transform=None):
        assert all(tensors[0].size(0) == tensor.size(0) for tensor in tensors)
        self.tensors = tensors
        self.labels  = labels
        self.transform = transform

    def __getitem__(self, index):
        x = self.tensors[index]

        if self.transform:
            x = self.transform(x)

        y = self.labels[index]

        return x, y

    def __len__(self):
        return self.tensors.size(0)

In [0]:
# what you want to actually modify, between the ToPILImage and ToTensor lines
train_augmentation = torchvision.transforms.Compose(
    [
        torchvision.transforms.ToPILImage(),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.ToTensor()
    ]
)

In [0]:
## To use augmentations:
trainset = CustomTensorDataset(
  tensors = training, 
  labels  = labels, 
  transform = train_augmentation
)

In [0]:
training[0].shape

torch.Size([3, 224, 224])

In [0]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True)

In [0]:
n_epochs = 5
optimizer = torch.optim.SGD(resnet18.parameters(), lr=0.1, momentum=0.9)

In [0]:
for epoch in range(n_epochs):
  for X_batch, Y_batch in trainloader:
    X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
    # Step 1: Zero out gradient
    optimizer.zero_grad()
    # Step 2: Compute Loss
    logits = resnet18(X_batch)
    loss = torch.nn.CrossEntropyLoss()(logits, Y_batch)
    #print("Loss:", loss.item())
    # Step 3: Backpropagation
    loss.backward()
    # Step 4: Update Weights
    optimizer.step()

In [0]:
torch.save(resnet18.state_dict(), '/content/drive/My Drive/resnet18-crossentropy-processed.pth')

In [0]:
test_labels = np.load('/content/drive/Shared drives/Data/Reprocessed data /Copy of TestAugLabels.npy',allow_pickle=True)
test_data = np.load('/content/drive/Shared drives/Data/Reprocessed data /Copy of TestDataAugData.npy',allow_pickle=True)

In [0]:
test_data = np.swapaxes(test_data,1,3)
test_data = np.swapaxes(test_data,2,3)

In [0]:
test_data = torch.from_numpy(test_data)

In [0]:
labels_temp = np.zeros(test_labels.shape[0])

for i in range(0,labels_temp.shape[0]): 
  if test_labels[i][0] == 1:
    labels_temp[i] = 1
  else:
    labels_temp[i] = 0

test_labels = torch.from_numpy(labels_temp)

test_labels

In [0]:
test_labels = test_labels.type(torch.LongTensor)
test_labels

In [0]:
# make test datasetloader
testset = torch.utils.data.TensorDataset(test_data,test_labels)

In [0]:
testloader = torch.utils.data.DataLoader(testset, batch_size=16, shuffle=True)

In [0]:
def test_accuracy(model, test_loader):

    # Do validation on the test set
    model.eval()
    model.to('cuda')

    with torch.no_grad():
    
        accuracy = 0
    
        for images, labels in iter(test_loader):
    
            images, labels = images.to('cuda'), labels.to('cuda')
    
            output = model.forward(images)

            probabilities = torch.exp(output)
        
            equality = (labels.data == probabilities.max(dim=1)[1])
        
            accuracy += equality.type(torch.FloatTensor).mean()
        
        print("Test Accuracy: {}".format(accuracy/len(test_loader)))    

In [0]:
test_accuracy(resnet18,testloader)

Test Accuracy: 0.8621794581413269
